In [1]:
from turtle import shape
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import scipy.io as spio

n_ch = 1000
h_ch = np.load(f'H6Length{n_ch}.npy')
ch_len = 5
h_sp = h_ch[ch_len//2][0:ch_len] 

h = np.zeros((ch_len,ch_len))

for i in range(ch_len//2+1):
    h[i][:ch_len//2 + i + 1] = h_sp[ch_len//2 - i:]
for i in range(ch_len//2+1):
    h[ch_len//2+i] = h[ch_len//2-i][::-1]
h

array([[ 0.84867432,  0.34356324, -0.14159457,  0.        ,  0.        ],
       [ 0.34356324,  0.84867432,  0.34356324, -0.14159457,  0.        ],
       [-0.14159457,  0.34356324,  0.84867432,  0.34356324, -0.14159457],
       [ 0.        , -0.14159457,  0.34356324,  0.84867432,  0.34356324],
       [ 0.        ,  0.        , -0.14159457,  0.34356324,  0.84867432]])

In [2]:
class cluster:
    def __init__(self,n_tran=17):
        self.n_tran = n_tran
        self.mid_tran = self.n_tran//2
        self.trun_len = 5
        self.mid_trun = self.trun_len//2
    
    def fitModel(self,neigh=1):
        pos = []
        neg = []
        # cnt = 0
        points = set()
        X = []
        Y = []
        self.s_in = []
        for i in range(2**self.n_tran):
            temp = 0
            sig = []
            for j in range(self.n_tran):
                if (i&(2**j)) != 0:
                    sig += [1]
                else: 
                    sig += [-1]
            o = []
            Y += [sig[self.mid_tran]]
            for j in range(self.mid_trun,self.n_tran - self.mid_trun):
                o += [np.round(np.dot(sig[j-self.mid_trun:j+self.mid_trun+1],h_sp),4)]
            X += [o]
            self.s_in += [sig[self.mid_trun:self.n_tran-self.mid_trun]]
            points.add(tuple(o)) 
            

            # if sig[self.mid_tran] < 0: 
            #     neg += [o[len(o)//2]]
            # else:
            #     pos += [o[len(o)//2]]
            

            temp = np.round(temp,4)
        

        print("number of cluster points:", len(points))
        self.X_train = np.array(X)
        self.Y_train = np.array(Y)
        pos = np.unique(pos)
        neg = np.unique(neg)
        print(*pos)
        # print(cnt)
        print("-*******-")
        print(*neg)



        print('cluster points shape',self.X_train.shape)
        self.model = KNeighborsClassifier(n_neighbors=neigh,weights='distance').fit(self.X_train,self.Y_train)
        # self.model = GaussianNB().fit(X,Y)
        print('error in training:', sum(self.model.predict(self.X_train) != self.Y_train))
        return self.model

    def reduced_model(self):
        
        return
    
    

    def testModelLLR(self,ebno,data):
        noisePw = 1/(10**(ebno/10))
        dim = self.n_tran - self.trun_len + 1
        for i in range(0,len(data),n_ch):
            print(i)
            s = data[i:i+n_ch]
            s = np.array(s).reshape((n_ch, 1))
            noise = np.sqrt(noisePw/2)*np.random.standard_normal((n_ch,1))
            r = np.dot(h_ch,s) + noise
            X_t = np.zeros((n_ch-(self.n_tran-1),dim))
            Y = []
            for i in range(self.mid_tran,n_ch-self.mid_tran):
                for id,val in enumerate(r[i-dim//2:i+dim//2+1]):
                    X_t[i-self.mid_tran][id] = np.round(val,4)
                Y += [1 if s[i] > 0 else -1]
            Y = np.array(Y)
            # Y_t = self.model.predict(X_t)
            
            self.X_test = X_t
            Y_t = self.model.kneighbors(self.X_test)
            
            return Y_t

    def testModel(self,ebno,num_iteration):
        noisePw = 1/(10**(ebno/10))
        dim = self.n_tran - self.trun_len + 1
        err = []
        for _ in range(num_iteration):
            print(_)
            s = 2*np.random.randint(2,size=(n_ch,1))-1
            noise = np.sqrt(noisePw/2)*np.random.standard_normal((n_ch,1))
            r = np.dot(h_ch,s) + noise
            X_t = np.zeros((n_ch-(self.n_tran-1),dim))
            Y = []
            for i in range(self.mid_tran,n_ch-self.mid_tran):
                for id,val in enumerate(r[i-dim//2:i+dim//2+1]):
                    X_t[i-self.mid_tran][id] = np.round(val,4)
                Y += [1 if s[i] > 0 else -1]
            Y = np.array(Y)
            
            Y_t = self.model.predict(X_t)
            err += [sum(Y_t != Y)/len(Y)]
        print(np.mean(err))
        return np.mean(err)


In [3]:
mat = spio.loadmat('./data/testData5e3_int.mat')
dataIn = np.array(mat['intrl'],dtype=int)[0]
print(len(dataIn))

10000


In [4]:

sysModel = cluster()
sysModel.fitModel(neigh=8)


number of cluster points: 131072

-*******-

cluster points shape (131072, 13)
error in training: 0


KNeighborsClassifier(n_neighbors=8, weights='distance')

In [5]:
y = sysModel.testModelLLR(4,dataIn)
print(y[0][0],y[1][0])

0
[1.76891809 1.77706433 1.78936284 1.84608321 1.8538904  1.86568255
 1.88531646 1.90006494] [ 48982  48983  48980 114518 114519 114516  49005  24406]


In [6]:
sysModel.X_test[0]

array([ 0.7581,  0.4657,  0.4023,  0.897 ,  0.2492,  0.767 , -0.0538,
        1.7072,  1.3214,  1.6914,  1.1917,  0.5446,  0.1894])

In [7]:
np.sqrt(sum(abs(sysModel.X_train[y[1][0][0]]- sysModel.X_test[0])**2))

1.7689180930727122

In [8]:
s_in = np.array(sysModel.s_in)
print(s_in.shape)
print(sysModel.X_train.shape)

(131072, 13)
(131072, 13)


In [9]:
from LLR import LLR

In [10]:

ebno = 4
noisePw = 1/(10**(ebno/10))
for i in range(len(y[0])):
    candList = []
    for idx in y[1][i]:
        candList += [s_in[idx].T]
    print(LLR(n = 13, candList = candList, noisePw=noisePw, dis=y[0][i]))

[[ 20.        ]
 [ -8.32689862]
 [  8.32689862]
 [ -8.32689862]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [  6.51999292]
 [ -6.51999292]]
[[ -2.49166568]
 [  3.94341306]
 [ -3.94341306]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]]
[[ 20.        ]
 [-10.32368759]
 [ 10.32368759]
 [-10.32368759]
 [ 10.32368759]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]]
[[  1.24688869]
 [  4.30116013]
 [ -4.30116013]
 [  4.30116013]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]]
[[ 20.]
 [-20.]
 [ 20.]
 [ 20.]
 [ 20.]
 [ 20.]
 [ 20.]
 [ 20.]
 [-20.]
 [ 20.]
 [ 20.]
 [ 20.]
 [ 20.]]
[[ -0.40020877]
 [  0.40020877]
 [ 20.        ]
 [ 20.     

[[ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [  5.94102448]
 [ -5.94102448]
 [-20.        ]]
[[ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [  3.08200928]
 [ -3.08200928]
 [ -3.13337411]
 [  3.13337411]
 [-20.        ]
 [-20.        ]]
[[ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [  5.56621964]
 [ -5.56621964]
 [ -3.18162389]
 [  3.18162389]
 [-20.        ]
 [-20.        ]
 [ 20.        ]]
[[-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [  5.62057215]
 [ -5.62057215]
 [  0.19180979]
 [ -0.19180979]
 [ -1.83837322]
 [-20.        ]
 [ 20.        ]
 [  3.79783012]]
[[ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [  5.33403268]
 [ -5.33403268]
 [ -2.05721555]
 [  2.05721555]
 [ -5.4934736 ]
 [-20.        ]
 [ 2

[[ -1.20119338]
 [  1.20119338]
 [  3.95907832]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [  3.51311868]
 [ -3.51311868]]
[[  2.85225147]
 [  4.87366806]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]]
[[ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ -3.26570688]]
[[ 20.        ]
 [ 20.        ]
 [  1.45132586]
 [ -4.49679465]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 10.12041823]
 [-10.12041823]
 [ 10.12041823]
 [-10.12041823]
 [ 20.        ]]
[[ 20.        ]
 [  0.75451612]
 [ -2.80751814]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [-2

[[ 20.        ]
 [-20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [  3.2879788 ]
 [ -3.2879788 ]
 [  3.2879788 ]
 [ -3.2879788 ]
 [  3.87151862]]
[[ -0.30019449]
 [  0.30019449]
 [ -0.30019449]
 [  0.30019449]
 [ -0.30019449]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]]
[[-20.]
 [ 20.]
 [-20.]
 [ 20.]
 [ 20.]
 [ 20.]
 [ 20.]
 [-20.]
 [ 20.]
 [-20.]
 [ 20.]
 [ 20.]
 [ 20.]]
[[  7.89370856]
 [ -7.89370856]
 [  7.89370856]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]]
[[ -0.51367813]
 [  0.51367813]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]]
[[  1.31429201]
 [ 20.        ]
 [ 20.        ]
 [ 20.     

[[ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [  3.11448087]
 [ 20.        ]
 [ 20.        ]
 [  3.11448087]
 [ 20.        ]
 [-20.        ]]
[[-20.       ]
 [ 20.       ]
 [ 20.       ]
 [ 20.       ]
 [ 20.       ]
 [ 20.       ]
 [  3.9398675]
 [ 20.       ]
 [  3.9398675]
 [ 20.       ]
 [ 20.       ]
 [-20.       ]
 [ 20.       ]]
[[20.       ]
 [20.       ]
 [20.       ]
 [20.       ]
 [20.       ]
 [20.       ]
 [20.       ]
 [20.       ]
 [20.       ]
 [ 7.805425 ]
 [-7.805425 ]
 [ 7.805425 ]
 [ 3.8785421]]
[[20.        ]
 [20.        ]
 [20.        ]
 [20.        ]
 [20.        ]
 [20.        ]
 [20.        ]
 [20.        ]
 [ 2.2395755 ]
 [-0.71470034]
 [ 0.71470034]
 [20.        ]
 [20.        ]]
[[ 20.        ]
 [ 20.        ]
 [  4.35507854]
 [  6.4432559 ]
 [ 20.        ]
 [ 20.        ]
 [  6.4432559 ]
 [ 10.25539432]
 [-10.25539432]
 [ 10.25539432]
 [ 20.        ]
 [ 20.        ]
 [ 10.25539432]]
[[20.  

...

[[ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ -5.02684612]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [-20.        ]
 [  4.97248789]]
[[ 20.        ]
 [ 20.        ]
 [ 19.172994  ]
 [-19.172994  ]
 [ 19.172994  ]
 [-19.172994  ]
 [ 20.        ]
 [ 19.172994  ]
 [-19.172994  ]
 [ 19.172994  ]
 [-19.172994  ]
 [  2.89678161]
 [ -2.89678161]]
[[ 20.        ]
 [ 21.71867654]
 [-21.71867654]
 [ 21.71867654]
 [-21.71867654]
 [ 20.        ]
 [ 21.71867654]
 [-21.71867654]
 [ 21.71867654]
 [-21.71867654]
 [ 21.71867654]
 [-21.71867654]
 [ 20.        ]]
[[ 0.38395052]
 [-0.38395052]
 [ 0.38395052]
 [-0.38395052]
 [20.        ]
 [ 9.36706576]
 [-7.35374677]
 [ 7.35374677]
 [-7.35374677]
 [ 7.35374677]
 [-7.35374677]
 [20.        ]
 [20.        ]]
[[ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ -0.2992302 ]
 [ 11.94895727]
 [-11.94895727]
 [ 11.94895727]
 [-11.94895727]
 [ 11.94895727]
 [ 20.        ]
 

[[ 2.00000000e+01]
 [ 2.00000000e+01]
 [ 2.00000000e+01]
 [-1.30217385e-02]
 [ 1.30217385e-02]
 [ 2.00000000e+01]
 [ 2.00000000e+01]
 [-2.00000000e+01]
 [ 2.00000000e+01]
 [-2.00000000e+01]
 [ 2.00000000e+01]
 [ 2.00000000e+01]
 [-2.00000000e+01]]
[[20.        ]
 [20.        ]
 [-0.83574644]
 [ 0.83574644]
 [20.        ]
 [20.        ]
 [-7.96872103]
 [ 7.96872103]
 [-7.96872103]
 [20.        ]
 [ 7.96872103]
 [-7.96872103]
 [ 7.96872103]]
[[ 20.        ]
 [  2.83542386]
 [ -2.83542386]
 [ 20.        ]
 [ 20.        ]
 [-10.2682785 ]
 [ 10.2682785 ]
 [-10.2682785 ]
 [ 20.        ]
 [ 10.2682785 ]
 [-10.2682785 ]
 [ 10.2682785 ]
 [-10.2682785 ]]
[[ -5.3547423 ]
 [  5.3547423 ]
 [ 20.        ]
 [ 20.        ]
 [  3.88063596]
 [ -3.88063596]
 [  3.88063596]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]]
[[  7.985044  ]
 [ 20.        ]
 [ 20.        ]
 [  5.60510696]
 [ -5.60510696]
 [  5.60510696]
 [ 20.        ]
 [ -4.80295734]
 [ 20.    

[[ 20.]
 [ 20.]
 [ 20.]
 [ 20.]
 [ 20.]
 [ 20.]
 [ 20.]
 [-20.]
 [ 20.]
 [ 20.]
 [ 20.]
 [ 20.]
 [ 20.]]
[[ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [  3.09511179]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [  4.92405703]
 [ 20.        ]]
[[ 20.        ]
 [ 20.        ]
 [  1.31441137]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]]
[[ 20.        ]
 [  0.82480347]
 [  3.9326627 ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]]
[[  0.7444305]
 [ 20.       ]
 [ 20.       ]
 [-20.       ]
 [ 20.       ]
 [ 20.       ]
 [ 20.       ]
 [ 20.       ]
 [ 20.       ]
 [ 20.       ]
 [ 20.       ]
 [ 20.       ]
 [-20.       ]]
[[ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.  

[[-11.67465885]
 [ 20.        ]
 [ 20.        ]
 [-11.67465885]
 [ 11.67465885]
 [-11.67465885]
 [ 11.67465885]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ -5.86108682]
 [  5.86108682]]
[[ 20.        ]
 [ 20.        ]
 [  0.38865992]
 [ -0.38865992]
 [  0.38865992]
 [ -0.38865992]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]]
[[ 20.        ]
 [  0.16682056]
 [ -0.16682056]
 [  0.16682056]
 [ -0.16682056]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [  4.32532433]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]]
[[  2.84120145]
 [ -2.84120145]
 [  2.84120145]
 [ -2.84120145]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]]
[[-20.]
 [ 20.]
 [-20.]
 [ 20.]
 [ 20.]
 [ 20.]
 [ 20.]
 [ 20.]
 [-20.]
 [ 20.]
 [ 20.]
 [-20.]
 [ 20.]]
[[ 20.]
 [-20.]
 [ 20.]
 [ 20.]
 [ 20.]
 [ 20.]
 [ 20.]
 [-

[[-20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ -2.12863154]
 [  2.12863154]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]]
[[ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ -0.79931251]
 [  0.79931251]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]]
[[ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ -1.94370343]
 [  1.94370343]
 [  4.70581529]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]]
[[-20.        ]
 [ 20.        ]
 [ 20.        ]
 [ 20.        ]
 [ -0.79931251]
 [  0.79931251]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]
 [ 20.        ]
 [-20.        ]
 [ 20.        ]]
[[ 20.       ]
 [ 20.       ]
 [ 20.       ]
 [ -2.0195495]
 [  2.0195495]
 [ 20.       ]
 [ 20.       ]
 [-20.       ]
 [ 20.       ]
 [ 20.       ]
 [-20.       ]